In [5]:
import numpy as np
import pandas as pd
from maze_env import Maze

class QLearningTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = actions  # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)
        print("init q_table ",self.q_table)

    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform() < self.epsilon:
            # choose best action
            state_action = self.q_table.loc[observation, :]
            state_action = state_action.reindex(np.random.permutation(state_action.index))     # some actions have same value
            action = state_action.idxmax()
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update
        print("Updated learnt q table",self.q_table)

    def check_state_exist(self, state):
        print("state ", state)
        print("self.q_table.index ", self.q_table.index)
        if state not in self.q_table.index:
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )
                    
if __name__ == "__main__":
    env = Maze()
    observation = env.reset()
    print("str(observation):",str(observation))
        
    action_space = ['u', 'd', 'l', 'r']
    n_actions = len(action_space)
    RL = QLearningTable(actions=list(range(n_actions)))
    
    action = RL.choose_action(str(observation))

    # RL take action and get next observation and reward
    observation_, reward, done = env.step(action)

    # RL learn from this transition
    RL.learn(str(observation), action, reward, str(observation_))
    print("RL.qtable:",RL.q_table)
    
    newSeries = pd.Series([0]*len(RL.actions),
              index=RL.q_table.columns,
              name='[5.0, 5.0, 35.0, 35.0]',
              )
    print("New Seriers:", newSeries)    
    
    

str(observation): [5.0, 5.0, 35.0, 35.0]
init q_table  Empty DataFrame
Columns: [0, 1, 2, 3]
Index: []
state  [5.0, 5.0, 35.0, 35.0]
self.q_table.index  Index([], dtype='object')
state  [5.0, 5.0, 35.0, 35.0]
self.q_table.index  Index(['[5.0, 5.0, 35.0, 35.0]'], dtype='object')
RL.qtable:                           0    1    2    3
[5.0, 5.0, 35.0, 35.0]  0.0  0.0  0.0  0.0
New Seriers: 0    0
1    0
2    0
3    0
Name: [5.0, 5.0, 35.0, 35.0], dtype: int64
